In [166]:
import pandas as pd
name = 'RELIANCE'

In [168]:
trade = pd.read_parquet(f'{name}_trade.parquet')
order = pd.read_parquet(f'{name}_order.parquet')

In [169]:
order.head(100000000000000)

,Timestamp,Side,Price,Volume_Disclosed,Price_Type,Symbol,Original_Qty,Order Number,IOC Flag,Activity Type
0,79631194602407,B,866.0,0.0,Limit,RELIANCE,1.0,1200000000001802,N,1
1,79631194602436,S,1055.0,0.0,Limit,RELIANCE,1.0,1200000000001804,N,1
2,79631194602723,S,960.0,0.0,Limit,RELIANCE,24.0,1200000000001826,N,1
3,79631194602855,B,950.0,0.0,Limit,RELIANCE,1.0,1200000000001835,N,1
4,79631194602868,B,950.0,0.0,Limit,RELIANCE,1.0,1200000000001836,N,1
...,...,...,...,...,...,...,...,...,...,...
1495403,79632724984054,None,NaN,NaN,None,None,NaN,1200000005328322,N,3
1495404,79632726947752,None,NaN,NaN,None,None,NaN,1200000005336801,N,3
1495405,79632724986868,None,NaN,NaN,None,None,NaN,1200000005341655,N,3
1495406,79632725516984,None,NaN,NaN,None,None,NaN,1200000005262718,N,3


In [170]:
trade.head()

,trade_time,price,quantity,buy_order_number,sell_order_number
5096248,79631223195796,965.0,1,1200000000056413,1200000000064315
5096249,79631223195797,965.0,1,1200000000056512,1200000000064315
5096250,79631223195798,965.0,10,1200000000010246,1200000000064315
5096251,79631223195799,965.0,3,1200000000039060,1200000000064315
5096252,79631223195800,965.0,47,1200000000031797,1200000000064315


In [5]:
avg = trade['price'].mean()


In [6]:
print(avg)

522.1893079019073


In [172]:
import pandas as pd
import csv

def create_quote_stream_corrected(
    order_log_path=f'{name}_order.parquet',
    trade_log_path=f'{name}_trade.parquet',
    quote_output_path=f'{name}_taq.csv'
):
    """
    Corrected version to process logs and create a quote stream.
    Implements robust "delete-then-add" logic for modified orders to prevent
    a corrupted live book and ensure all events are processed correctly.
    """
    # --- Step 1: Loading and Preparing Data ---
    print("--- Step 1: Loading and Preparing Data ---")
    try:
        order_df = pd.read_parquet(order_log_path)
        trade_df = pd.read_parquet(trade_log_path)
        
        for col in ['Order Number', 'buy_order_number', 'sell_order_number']:
            if col in order_df.columns: order_df[col] = order_df[col].astype(str)
            if col in trade_df.columns: trade_df[col] = trade_df[col].astype(str)
    except FileNotFoundError as e:
        print(f"\nError: Data file not found. Make sure '{e.filename}' is in the correct directory.")
        return

    order_df_copy = order_df[order_df['Price_Type'] != 'Trigger'].copy()
    order_df_copy.rename(columns={'Timestamp': 'timestamp', 'Order Number': 'order_number'}, inplace=True)
    def activity_to_event(act_type):
        if act_type == 3: return 'CANCEL'
        if act_type == 4: return 'MODIFY'
        return 'NEW'
    order_df_copy['event_type'] = order_df_copy['Activity Type'].apply(activity_to_event)
    order_events = order_df_copy[['timestamp', 'event_type', 'order_number', 'Side', 'Price', 'Volume_Disclosed', 'Original_Qty']]

    trade_df_copy = trade_df.copy()
    trade_df_copy.rename(columns={'trade_time': 'timestamp'}, inplace=True)
    trade_df_copy['event_type'] = 'TRADE'
    trade_events = trade_df_copy[['timestamp', 'event_type', 'price', 'quantity', 'buy_order_number', 'sell_order_number']]

    # --- Step 2: Combining and Sorting All Market Events ---
    
    print("\n--- Step 2: Combining and Sorting All Market Events ---")
    all_events = pd.concat([order_events, trade_events], ignore_index=True, sort=False)
    all_events.sort_values(by='timestamp', inplace=True)
    all_events.reset_index(drop=True, inplace=True)
    total_events = len(all_events)
    print(f"Total chronological events to process: {total_events}")

    # --- Step 3: Reconstructing Order Book with Corrected Logic ---
    live_book = {}
    last_traded_price = 0.0
    flag = 0
    
    with open(quote_output_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['timestamp', 'best_bid', 'depth_bid', 'best_bid_ud', 'depth_bid_ud',  'best_ask', 'depth_ask', 'best_ask_ud', 'depth_ask_ud', 'ltp'])

        print("\n--- Step 3: Reconstructing Order Book and Generating Quote Stream ---")
        for index, event in all_events.iterrows():
            timestamp = event['timestamp']

            
            # Part A: Update Live Book
            if event['event_type'] == 'NEW' and event['Original_Qty'] >= 1e-6:

                live_book[event['order_number']] = {
                    'side': event['Side'], 'price': event['Price'], 'quantity': event['Original_Qty'], 'disclosed' : event['Volume_Disclosed'], 'initial_d' : event['Volume_Disclosed']
                }
            
            # *** CORRECTED MODIFICATION LOGIC ***
            elif event['event_type'] == 'MODIFY':
                order_num = event['order_number']
                volume = event['Original_Qty']
                if order_num in live_book:
                    del live_book[order_num]

                if volume >= 1e-6:
                    live_book[order_num] = {
                      'side': event['Side'], 'price': event['Price'], 'quantity': event['Original_Qty'], 'disclosed' : event['Volume_Disclosed'], 'initial_d' : event['Volume_Disclosed']
                }

            elif event['event_type'] == 'CANCEL':
                if event['order_number'] in live_book:
                    del live_book[event['order_number']]

            elif event['event_type'] == 'TRADE':
                if not live_book:
                    best_bid, best_ask, best_bid_ud, best_ask_ud = 0.0, 0.0, 0.0, 0.0
                else:
                    bids = [order['price'] for order in live_book.values() if order['side'] == 'B' and order['disclosed'] >= 1e-6]
                    asks = [order['price'] for order in live_book.values() if order['side'] == 'S' and order['disclosed'] >= 1e-6]
                    best_bid = max(bids) if bids else 0.0
                    best_ask = min(asks) if asks else 0.0
                    bids = [order['price'] for order in live_book.values() if order['side'] == 'B']
                    asks = [order['price'] for order in live_book.values() if order['side'] == 'S']
                    best_bid_ud = max(bids) if bids else 0.0
                    best_ask_ud = min(asks) if asks else 0.0

               
                bid_depth = sum(o['disclosed'] for o in live_book.values() if o['side'] == 'B' and o['price'] == best_bid)
                ask_depth = sum(o['disclosed'] for o in live_book.values() if o['side'] == 'S' and o['price'] == best_ask)
                depth_bid_ud = sum(o['quantity'] for o in live_book.values() if o['side'] == 'B' and o['price'] == best_bid_ud)
                depth_ask_ud = sum(o['quantity'] for o in live_book.values() if o['side'] == 'S' and o['price'] == best_ask_ud)

                    
                last_traded_price = event['price']
                trade_qty = event['quantity']
                for order_num_hit in [event['buy_order_number'], event['sell_order_number']]:
                    if order_num_hit in live_book:
                        initial_disclosed_size = live_book[order_num_hit]['initial_d']
                        live_book[order_num_hit]['quantity'] -= trade_qty
                        live_book[order_num_hit]['disclosed'] -= trade_qty
                        if live_book[order_num_hit]['quantity'] <= 1e-6:
                            del live_book[order_num_hit]
                        elif live_book[order_num_hit]['disclosed'] <= 1e-6:
                            remaining_qty = live_book[order_num_hit]['quantity']
                            live_book[order_num_hit]['disclosed'] = min(initial_disclosed_size, remaining_qty)

                writer.writerow([timestamp, best_bid, bid_depth, best_bid_ud, depth_bid_ud,  best_ask, ask_depth, best_ask_ud, depth_ask_ud, last_traded_price])

        
            if (index + 1) % 100000 == 0 or (index + 1) == total_events:
                progress_pct = ((index + 1) / total_events) * 100
                print(f"  Processed {index + 1} / {total_events} events ({progress_pct:.1f}%)", end='\r')

    print(f"\n\nProcessing complete! Quote stream saved to '{quote_output_path}'. 🚀")

if __name__ == '__main__':
    create_quote_stream_corrected()

--- Step 1: Loading and Preparing Data ---



--- Step 2: Combining and Sorting All Market Events ---
Total chronological events to process: 1607508

--- Step 3: Reconstructing Order Book and Generating Quote Stream ---
  Processed 1607508 / 1607508 events (100.0%)

Processing complete! Quote stream saved to 'RELIANCE_taq.csv'. 🚀


In [173]:
trade_q = trade['quantity'].reset_index()
trade_q = trade_q.drop('index', axis = 1)

trade_q.head()

,quantity
0,1
1,1
2,10
3,3
4,47


In [174]:
len(trade_q)

117456

In [175]:
dff = pd.read_csv(f'{name}_taq.csv')

In [176]:
count = ((dff['ltp'] != dff['best_bid_ud']) & (dff['ltp'] != dff['best_ask_ud'])).sum()
print(count, len(dff), "mismatched = ", 100 * count / len(dff), '%')

3248 117456 mismatched =  2.765290832311674 %


In [177]:
count = (dff['best_bid'] > dff['best_ask']).sum()
print(count, len(dff), "mismatched = ", 100 * count / len(dff), '%')

4348 117456 mismatched =  3.7018117422694456 %


In [178]:
count = (dff['best_bid_ud'] > dff['best_ask_ud']).sum()
print(count, len(dff), "mismatched = ", 100 * count / len(dff), '%')

38895 117456 mismatched =  33.11452799346138 %


In [179]:
count = ((dff['ltp'] > dff['best_ask']) | ((dff['ltp'] < dff['best_bid']))).sum()
print(count, len(dff), "mismatched = ", 100 * count / len(dff), '%')

7367 117456 mismatched =  6.27213594878082 %


In [101]:

mask = (dff['best_bid_ud'] > dff['best_ask_ud']) & ((dff['ltp'] == dff['best_ask_ud']))

change_points = mask.ne(mask.shift()).cumsum()

mismatch_groups = dff[mask].groupby(change_points[mask])

results = []

for _, group in mismatch_groups:
    start_idx = group.index[0]
    end_idx = group.index[-1]
    duration = dff.loc[end_idx, 'timestamp'] - dff.loc[start_idx, 'timestamp']
    results.append({
        'start_index': start_idx,
        'end_index': end_idx,
        'length': len(group),
        'duration': duration
    })

results_df = pd.DataFrame(results)

print(results_df)
print("\nLongest mismatch sequence:")
print(results_df.loc[results_df['length'].idxmax()])

# Top 5 mismatch sequences by length
top5 = results_df.sort_values(by='length', ascending=False).head(5)
print(top5)



      start_index  end_index  length  duration
0             133        135       3     23524
1             141        142       2         4
2             145        148       4     29780
3             156        156       1         0
4             191        191       1         0
...           ...        ...     ...       ...
4766       140639     140639       1         0
4767       140657     140661       5         4
4768       140678     140690      13        12
4769       140707     140707       1         0
4770       140720     140722       3         2

[4771 rows x 4 columns]

Longest mismatch sequence:
start_index    135330
end_index      135489
length            160
duration          159
Name: 4613, dtype: int64
      start_index  end_index  length  duration
4613       135330     135489     160       159
3638       108100     108244     145       501
660         14379      14489     111       110
2734        66978      67088     111       112
4308       123475     123567      9

In [222]:
best_bid_max = dff['best_bid']
best_ask_min = dff['best_ask']

mask = best_bid_max > best_ask_min
count = mask.sum()

print(count, len(dff), "mismatched =", 100 * count / len(dff), '%')


4348 117456 mismatched = 3.7018117422694456 %


In [180]:
print(len(dff) == len(trade_q))

True


In [181]:
dff = dff.merge(trade_q, left_index=True, right_index=True, how='inner')
len(dff)


117456

In [182]:
dff.head(60006)

,timestamp,best_bid,depth_bid,best_bid_ud,depth_bid_ud,best_ask,depth_ask,best_ask_ud,depth_ask_ud,ltp,quantity
0,79631223195796,0.00,0.0,1025.00,1.0,0.00,0.0,864.55,62.0,965.00,1
1,79631223195797,0.00,0.0,1020.00,1.0,0.00,0.0,864.55,61.0,965.00,1
2,79631223195798,0.00,0.0,1000.00,13.0,0.00,0.0,864.55,60.0,965.00,10
3,79631223195799,0.00,0.0,1000.00,3.0,0.00,0.0,864.55,50.0,965.00,3
4,79631223195800,0.00,0.0,999.00,124.0,0.00,0.0,864.55,47.0,965.00,47
...,...,...,...,...,...,...,...,...,...,...,...
60001,79631892960276,973.15,1.0,973.25,88.0,973.25,20.0,973.25,58.0,973.25,5
60002,79631892960277,973.15,1.0,973.25,83.0,973.25,17.0,973.25,53.0,973.25,4
60003,79631892960278,973.15,1.0,973.25,79.0,973.25,17.0,973.25,49.0,973.25,11
60004,79631892960279,973.15,1.0,973.25,68.0,973.25,17.0,973.25,38.0,973.25,2


In [183]:
dff.to_parquet(f'{name}_taq.parquet')

117456

FOR TESTING

--------------------------------------------------------

In [87]:
try:
    trade_df = pd.read_parquet(f'{name}_trade.parquet')
    order_df = pd.read_parquet(f'{name}_order.parquet')
    
    for col in ['Order Number', 'buy_order_number', 'sell_order_number']:
        if col in order_df.columns: order_df[col] = order_df[col].astype(str)
        if col in trade_df.columns: trade_df[col] = trade_df[col].astype(str)
except FileNotFoundError as e:
    print(f"\nError: Data file not found. Make sure '{e.filename}' is in the correct directory.")

order_df_copy = order_df[order_df['Price_Type'] != 'Trigger'].copy()
order_df_copy.rename(columns={'Timestamp': 'timestamp', 'Order Number': 'order_number'}, inplace=True)
def activity_to_event(act_type):
    if act_type == 3: return 'CANCEL'
    if act_type == 4: return 'MODIFY'
    return 'NEW'
order_df_copy['event_type'] = order_df_copy['Activity Type'].apply(activity_to_event)
order_events = order_df_copy[['timestamp', 'event_type', 'order_number', 'Side', 'Price', 'Volume_Disclosed', 'Original_Qty']]

trade_df_copy = trade_df.copy()
trade_df_copy.rename(columns={'trade_time': 'timestamp'}, inplace=True)
trade_df_copy['event_type'] = 'TRADE'
trade_events = trade_df_copy[['timestamp', 'event_type', 'price', 'quantity', 'buy_order_number', 'sell_order_number']]

# --- Step 2: Combining and Sorting All Market Events ---

print("\n--- Step 2: Combining and Sorting All Market Events ---")
all_events = pd.concat([order_events, trade_events], ignore_index=True, sort=False)
all_events.sort_values(by='timestamp', inplace=True)
all_events.reset_index(drop=True, inplace=True)
total_events = len(all_events)
print(f"Total chronological events to process: {total_events}")


--- Step 2: Combining and Sorting All Market Events ---
Total chronological events to process: 1513022


In [ ]:
print((int(all_events.iloc[40000]['timestamp']) - int(all_events.iloc[0]['timestamp'])))

396727692


In [102]:
fil = all_events[(all_events['order_number'] == '1100000003187114') | (all_events['sell_order_number'] == '1100000003187114')]
fil.head(160)

,timestamp,event_type,order_number,Side,Price,Volume_Disclosed,Original_Qty,price,quantity,buy_order_number,sell_order_number
815423,79631953610178,NEW,1100000003187114,S,1343.5,500.0,5000.0,NaN,NaN,NaN,NaN
815424,79631953610179,TRADE,NaN,NaN,NaN,NaN,NaN,1343.8,289.0,1100000003184302,1100000003187114
815425,79631953610180,TRADE,NaN,NaN,NaN,NaN,NaN,1343.8,1.0,1100000003186183,1100000003187114
815426,79631953610181,TRADE,NaN,NaN,NaN,NaN,NaN,1343.8,2.0,1100000003186185,1100000003187114
815427,79631953610182,TRADE,NaN,NaN,NaN,NaN,NaN,1343.8,1.0,1100000003186603,1100000003187114
...,...,...,...,...,...,...,...,...,...,...,...
824318,79631958114317,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,100.0,1100000001379753,1100000003187114
824319,79631958114318,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,10.0,1100000002014139,1100000003187114
824320,79631958114319,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,1.0,1100000002339465,1100000003187114
824321,79631958114320,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,100.0,1100000001379753,1100000003187114


In [99]:
fil = all_events[(all_events['order_number'] == '1100000003198775') | (all_events['buy_order_number'] == '1100000003198775')]
fil.head(1000000)

,timestamp,event_type,order_number,Side,Price,Volume_Disclosed,Original_Qty,price,quantity,buy_order_number,sell_order_number
824202,79631958114206,NEW,1100000003198775,B,1340.1,0.0,28.0,NaN,NaN,NaN,NaN
824338,79631958114338,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,28.0,1100000003198775,1100000003187114


In [23]:
fil = all_events[(all_events['order_number'] == '1100000002707819')]
fil.head(100000000000)

,timestamp,event_type,order_number,Side,Price,Volume_Disclosed,Original_Qty,price,quantity,buy_order_number,sell_order_number
721948,79631792963692,NEW,1100000002707819,B,1345.6,37.0,37.0,NaN,NaN,NaN,NaN


In [98]:
fil = all_events[(all_events['price'] == 1340) & (all_events['timestamp'] <= '79631958114338')]
fil.head(1000000)

,timestamp,event_type,order_number,Side,Price,Volume_Disclosed,Original_Qty,price,quantity,buy_order_number,sell_order_number
14931,79631257443715,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,10.0,1100000000142304,1100000000008530
14932,79631257443716,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,5.0,1100000000142304,1100000000019184
14933,79631257443717,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,50.0,1100000000142304,1100000000019823
14934,79631257443718,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,2.0,1100000000142304,1100000000021808
14935,79631257443719,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,4.0,1100000000142304,1100000000031952
...,...,...,...,...,...,...,...,...,...,...,...
824334,79631958114332,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,100.0,1100000001379753,1100000003187114
824335,79631958114333,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,10.0,1100000002014139,1100000003187114
824336,79631958114334,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,100.0,1100000001379753,1100000003187114
824337,79631958114335,TRADE,NaN,NaN,NaN,NaN,NaN,1340.0,10.0,1100000002014139,1100000003187114


In [44]:
print((count)) 

6547


In [192]:
filtered_order = order[(order['Timestamp'] <= '79631425757699') & (order['Activity Type'] != 3) & (order['Side'] == 'S') & (order['Price'] >= 1348.5) & (order['Price'] <= 1349) ]

In [193]:
filtered_order.head(10500)

,Timestamp,Side,Price,Volume_Disclosed,Price_Type,Symbol,Original_Qty,Order Number,IOC Flag,Activity Type
69,79631195169485,S,1349.0,0.0,Limit,INFY,100.0,1100000000010602,N,1
319,79631197838593,S,1349.0,0.0,Limit,INFY,6.0,1100000000027845,N,1
361,79631198179500,S,1349.0,0.0,Limit,INFY,100.0,1100000000029771,N,1
365,79631198245028,S,1349.0,0.0,Limit,INFY,5.0,1100000000030091,N,1
396,79631198706103,S,1349.0,0.0,Limit,INFY,6.0,1100000000031756,N,1
...,...,...,...,...,...,...,...,...,...,...
338251,79631423289750,S,1348.9,0.0,Limit,INFY,25.0,1100000001295829,N,1
340536,79631424332332,S,1349.0,0.0,Limit,INFY,4.0,1100000001300526,N,1
340680,79631424594887,S,1349.0,0.0,Limit,INFY,20.0,1100000001302064,N,1
340841,79631424929965,S,1349.0,0.0,Limit,INFY,500.0,1100000001303713,N,1


In [ ]:
filtered_trade = trade[trade[]]

In [99]:
# Max timestamp of MODIFY per order
modify_times = all_events[all_events['event_type'] == 'MODIFY'].groupby('order_number')['timestamp'].max()

# Min timestamp of TRADE per order (either as buyer or seller)
trade_buy = all_events[all_events['event_type'] == 'TRADE'].groupby('buy_order_number')['timestamp'].min()
trade_sell = all_events[all_events['event_type'] == 'TRADE'].groupby('sell_order_number')['timestamp'].min()

# Combine both buy and sell trade times
trade_times = pd.concat([trade_buy, trade_sell]).groupby(level=0).min()

# Common orders between MODIFY and TRADE
common_orders = modify_times.index.intersection(trade_times.index)

# Compare timestamps
condition = trade_times.loc[common_orders] > modify_times.loc[common_orders]

# Results
result = pd.DataFrame({
    'modify_max': modify_times.loc[common_orders],
    'trade_min': trade_times.loc[common_orders],
    'trade_after_modify': condition
})

result = result[result['trade_after_modify'] == False]

print(result)


                      modify_max       trade_min  trade_after_modify
1100000000069572  79631254723971  79631253520683               False
1100000000072151  79631253954499  79631253897221               False
1100000000073757  79631254175724  79631253790249               False
1100000000074171  79631254458777  79631254438186               False
1100000000076667  79631253652531  79631253647854               False
...                          ...             ...                 ...
1100000006022132  79632723673937  79632722302055               False
1100000006022138  79632722166807  79632720652112               False
1100000006027291  79632723067473  79632722436063               False
1100000006031422  79632725096058  79632723995001               False
1100000006035527  79632727605010  79632725001587               False

[5066 rows x 3 columns]


In [108]:
result.head(500)

,modify_max,trade_min,trade_after_modify
1100000000069572,79631254723971,79631253520683,False
1100000000072151,79631253954499,79631253897221,False
1100000000073757,79631254175724,79631253790249,False
1100000000074171,79631254458777,79631254438186,False
1100000000076667,79631253652531,79631253647854,False
...,...,...,...
1100000000536050,79631302632837,79631302520181,False
1100000000539395,79631299190043,79631298329197,False
1100000000541230,79631302663870,79631302520182,False
1100000000541241,79631299596336,79631299047067,False


In [97]:
false_count = (~result['trade_after_modify']).sum()
print("Total false:", false_count)


Total false: 5066


In [111]:
fil = all_events[(all_events['order_number'] == '1100000000539395') | (all_events['sell_order_number'] == '1100000000539395')]
fil.head(1000000)

,timestamp,event_type,order_number,Side,Price,Volume_Disclosed,Original_Qty,price,quantity,buy_order_number,sell_order_number
127099,79631298317195,NEW,1100000000539395,S,1337.60,105.0,105.0,NaN,NaN,NaN,NaN
127113,79631298317211,MODIFY,1100000000539395,S,1337.65,105.0,105.0,NaN,NaN,NaN,NaN
127131,79631298329197,TRADE,NaN,NaN,NaN,NaN,NaN,1337.65,6.0,1100000000539491,1100000000539395
127133,79631298329203,MODIFY,1100000000539395,S,1338.65,105.0,105.0,NaN,NaN,NaN,NaN
127806,79631298924549,MODIFY,1100000000539395,S,1337.65,105.0,105.0,NaN,NaN,NaN,NaN
128041,79631299101002,TRADE,NaN,NaN,NaN,NaN,NaN,1337.65,72.0,1100000000546124,1100000000539395
128146,79631299167060,MODIFY,1100000000539395,S,1338.65,105.0,105.0,NaN,NaN,NaN,NaN
128310,79631299190043,MODIFY,1100000000539395,S,1337.65,105.0,105.0,NaN,NaN,NaN,NaN
129045,79631299441394,CANCEL,1100000000539395,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [105]:
fil = all_events[(all_events['order_number'] == '1100000003184302') | (all_events['buy_order_number'] == '1100000003184302')]
fil.head(1000000)

,timestamp,event_type,order_number,Side,Price,Volume_Disclosed,Original_Qty,price,quantity,buy_order_number,sell_order_number
815063,79631952509960,NEW,1100000003184302,B,1343.8,0.0,300.0,NaN,NaN,NaN,NaN
815389,79631953274612,TRADE,NaN,NaN,NaN,NaN,NaN,1343.8,6.0,1100000003184302,1100000003186197
815416,79631953572806,TRADE,NaN,NaN,NaN,NaN,NaN,1343.8,5.0,1100000003184302,1100000003187032
815424,79631953610179,TRADE,NaN,NaN,NaN,NaN,NaN,1343.8,289.0,1100000003184302,1100000003187114


In [115]:
fil = all_events[(all_events['order_number'] == '1100000000543266') | (all_events['sell_order_number'] == '1100000000543266')]
fil.head(1000000)

,timestamp,event_type,order_number,Side,Price,Volume_Disclosed,Original_Qty,price,quantity,buy_order_number,sell_order_number
127741,79631298797058,NEW,1100000000543266,S,1337.65,2.0,12.0,NaN,NaN,NaN,NaN
127949,79631298954477,MODIFY,1100000000543266,S,1337.65,1.0,6.0,NaN,NaN,NaN,NaN
128040,79631299101001,TRADE,NaN,NaN,NaN,NaN,NaN,1337.65,1.0,1100000000546124,1100000000543266
128049,79631299111504,MODIFY,1100000000543266,S,1337.65,1.0,2.0,NaN,NaN,NaN,NaN
128473,79631299203410,MODIFY,1100000000543266,S,1337.65,0.0,1.0,NaN,NaN,NaN,NaN
128930,79631299360678,CANCEL,1100000000543266,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [253]:
order1 = order[(order['Price_Type'] == 'Trigger') & (order['Activity Type'] == 4)]
order1 = order1['Order Number']
order1.head()

14211    1100000000107232
15446    1100000000130120
18050    1100000000136201
22665    1100000000070097
26328    1100000000126439
Name: Order Number, dtype: object

In [81]:
order2 = order[order['Order Number'] == '1100000003198575']
order2.head()

,Timestamp,Side,Price,Volume_Disclosed,Price_Type,Symbol,Original_Qty,Order Number,IOC Flag,Activity Type
729722,79631958042526,S,1340.45,0.0,Trigger,INFY,100.0,1100000003198575,N,1
729723,79631958042527,S,1340.45,0.0,Trigger,INFY,100.0,1100000003198575,N,1


In [262]:
for o in order1:
    order2 = order[order['Order Number'] == o]
    for j in order2['Price_Type']:
        
        if j == "Limit":
            print(o)

1100000000126439
1100000000126439
1100000000237812
1100000000100669
1100000000071393
1100000000668515
1100000000695042
1100000000824758
1100000000824758
1100000000824758
1100000000830352
1100000000830352
1100000001233612
1100000001143294
1100000001200235
1100000001200235
1100000000218681
1100000000218681
1100000001683837
1100000001683837
1100000001683837
1100000001200235
1100000000438457
1100000001487073
1100000001276926
1100000001217674
1100000001217674


KeyboardInterrupt: 

In [103]:
fil = all_events[(all_events['order_number'] == '1100000003187114') | (all_events['sell_order_number'] == '1100000003187114')]
fil.to_csv('fil.csv')